In [437]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd

In [438]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

In [439]:
# Product Demand
demand = np.random.randint(0,20,(Products,Market))

In [440]:
# Cost of opening
f_i = [20, 25]
f_j = [15, 5, 10]

In [441]:
# Unit cost of manufacturing product 
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [442]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,5, (Products, Manufacturing_plants, Distribution))

In [443]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,5, (Products, Distribution, Market))

In [444]:
# Plant Capacities
Capacities_i = np.random.randint(300,500,(Manufacturing_plants)) # in volume !!! (metres cubed)
Capacities_j = np.random.randint(300,500,(Distribution)) # in volume !!! (metres cubed)
Capacities_l = np.zeros((Products,Outsourced)) # in terms of products 
np.fill_diagonal(Capacities_l, random.randint(15,20))

In [445]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
Supplier_cost = np.zeros((Products, Outsourced))
np.fill_diagonal(Supplier_cost, random.uniform(10,15))

In [446]:
# Cost of transporting product m from supplier to j
T_O_DC = np.zeros((Products, Outsourced, Distribution))
T_O_DC[0][0] = np.random.uniform(10,15,(Distribution))
T_O_DC[1][1] = np.random.uniform(10,15,(Distribution))

In [447]:
# Cost of shipping product m from supplier to k
T_O_MZ = np.zeros((Products, Outsourced, Market))
T_O_MZ[0][0] = np.random.uniform(15,20,(Market))
T_O_MZ[1][1] = np.random.uniform(15,20,(Market))

In [448]:
# Product volume 
volume = np.random.randint(2,5,(Products))

In [449]:
# unit cost of lost sales 
lost_sales = np.random.randint(200,300,(Market,Products))

In [450]:
## Model

In [451]:
grbModel = Model('synthetic')

In [452]:
a_i = np.ones(Manufacturing_plants)
b_j = np.ones(Distribution)

In [453]:
# Model Variables

x_i = grbModel.addVars(range(Manufacturing_plants), vtype = GRB.BINARY) # opening manufacturing plant
x_j = grbModel.addVars(range(Distribution), vtype = GRB.BINARY) # opening DC
U_km = grbModel.addVars(range(Market), range(Products), vtype = GRB.INTEGER) # quantity lost sales
V_lm = grbModel.addVars(range(Products), range(Outsourced), vtype = GRB.INTEGER) # quantity products purchased from outsourcing
Q_im = grbModel.addVars(range(Products), range(Manufacturing_plants), vtype = GRB.INTEGER) # quantity produced
Y_ijm = grbModel.addVars(range(Products), range(Manufacturing_plants), range(Distribution), vtype = GRB.INTEGER) # shipping i -> j
Z_jkm = grbModel.addVars(range(Products), range(Distribution), range(Market), vtype = GRB.INTEGER) # shipping j -> k
T_ljm = grbModel.addVars(range(Products), range(Outsourced), range(Distribution), vtype = GRB.INTEGER) # shipping l -> j
T_lkm = grbModel.addVars(range(Products), range(Outsourced), range(Market), vtype = GRB.INTEGER) # shipping l -> k


In [454]:
# Model Constraints
# Network Flow
#grbModel.addConstr(quicksum(x_i[i] for i in range(Manufacturing_plants)) >= 1)
#grbModel.addConstr(quicksum(x_j[j] for i in range(Distribution)) >= 1)

grbModel.addConstrs(quicksum(Y_ijm[m,i,j] for j in range(Distribution)) == Q_im[m,i] 
             for i in range(Manufacturing_plants) for m in range(Products))

grbModel.addConstrs(quicksum(Z_jkm[m,j,k] for k in range(Market)) == 
                      (quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) + 
                       quicksum(T_ljm[m,l,j] for l in range(Outsourced))) for j in range(Distribution) 
                      for m in range(Products))

grbModel.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) + quicksum(T_lkm[m,l,k] for l in range(Outsourced))) ==
                      (demand[m][k] - U_km[k,m]) 
                    for k in range(Market) for m in range(Products))


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>}

In [455]:
for m in range(Products):
    for l in range(Outsourced):
        if l != m:
            grbModel.addConstr(V_lm[m,l] == 0)

In [456]:
for m in range(Products):
    for l in range(Outsourced):
        for j in range(Distribution):
            if l != m:
                grbModel.addConstr(T_ljm[m,l,j] == 0)

In [457]:
for m in range(Products):
    for l in range(Outsourced):
        for k in range(Market):
            if l != m:
                grbModel.addConstr(T_lkm[m,l,k] == 0)

In [458]:
# Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
grbModel.addConstrs(V_lm[m,l] == quicksum(T_ljm[m,l,j] for j in range(Distribution)) + quicksum(T_lkm[m,l,k] for k in range(Market))
            for m in range(Products) for l in range(Outsourced))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>}

In [459]:
# Capacity Constraints
grbModel.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= a_i[i]*Capacities_i[i]*x_i[i] for i in range(Manufacturing_plants))
grbModel.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
            quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= b_j[j]*Capacities_j[j]*x_j[j] 
            for j in range(Distribution))
grbModel.addConstrs(quicksum(V_lm[m,l] for m in range(Products)) <= np.sum(Capacities_l[l]) for l in range(Outsourced))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [460]:
# Objective
grb_expr = LinExpr()

# Cost of opening
OC_1 = 0
OC_2 = 0
for i in range(Manufacturing_plants):
    OC_1 += f_i[i]*x_i[i]
for j in range(Distribution):
    OC_2 += f_j[j]*x_j[j]
    
#grb_expr += OC_1 + OC_2
ship_1 = 0
ship_2 = 0
ship_3 = 0
ship_4 = 0

# Shipment
for i in range(Manufacturing_plants):
    for j in range(Distribution):
        for m in range(Products):
            ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]
            
for j in range(Distribution):
    for k in range(Market):
        for m in range(Products):
            ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

for l in range(Outsourced):
    for j in range(Distribution):
        for m in range(Products):
            ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

for l in range(Outsourced):
    for k in range(Market):
        for m in range(Products):
            ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]

#grb_expr += ship_1 + ship_2 + ship_3 + ship_4

# Production
pr_cost = 0
for i in range(Manufacturing_plants):
    for m in range(Products):
        pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]

#grb_expr += pr_cost
# Buying from outsource cost
b_cost = 0
for l in range(Outsourced):
    for m in range(Products):
        b_cost += Supplier_cost[m][l]*V_lm[m,l]
#grb_expr += b_cost

#Lost Sales
l_cost = 0
for k in range(Market):
    for m in range(Products):
        l_cost += lost_sales[k][m]*U_km[k,m]

grb_expr += OC_1 + OC_2 + ship_1 + ship_2 + ship_3 + ship_4 + pr_cost + b_cost + l_cost
        

In [461]:
grbModel.setObjective(grb_expr, GRB.MINIMIZE)

In [462]:
grbModel.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 45 rows, 85 columns and 197 nonzeros
Model fingerprint: 0x4c1ec2d0
Variable types: 0 continuous, 85 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [5e-02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+01]
Found heuristic solution: objective 14433.000000
Presolve removed 24 rows and 30 columns
Presolve time: 0.00s
Presolved: 21 rows, 55 columns, 123 nonzeros
Variable types: 0 continuous, 55 integer (5 binary)

Root relaxation: objective 1.675104e+02, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  167.51042    0    4 14433.0000  167.51042  98.8%     -    0s
H    0     0                     215.6253627  167.51042  22.3%     -    0s
H    0     0                     204.7829303  167.51042  18.2%   

In [463]:
v_val_x_i = grbModel.getAttr('x', x_i)
v_val_x_j = grbModel.getAttr('x', x_j)
v_val_U_km = grbModel.getAttr('x', U_km)
v_val_V_lm = grbModel.getAttr('x', V_lm)
v_val_Q_im = grbModel.getAttr('x', Q_im)
v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
v_val_T_ljm = grbModel.getAttr('x', T_ljm)
v_val_T_lkm = grbModel.getAttr('x', T_lkm)

In [464]:
obj = grbModel.getObjective()
print("obj val: ", obj.getValue())
print("Open Manufacturing Plants: ", np.sum(v_val_x_i.values()))
print("Open Distribution Centres: ", np.sum(v_val_x_j.values()))
print("Total purchased from Outsourcing: ", np.sum(v_val_V_lm.values()))
print("Total produced in house: ", np.sum(v_val_Q_im.values()))
print("Total lost sales in units: ", np.sum(v_val_U_km.values()))
print("Total shipped from outsourced facility to DCs: ", np.sum(v_val_T_ljm.values()))
print("Total shipped from outsourced facility to market zone: ", np.sum(v_val_T_lkm.values()))

obj val:  201.54084325318277
Open Manufacturing Plants:  1.0
Open Distribution Centres:  2.0
Total purchased from Outsourcing:  0.0
Total produced in house:  54.0
Total lost sales in units:  0.0
Total shipped from outsourced facility to DCs:  0.0
Total shipped from outsourced facility to market zone:  0.0


In [465]:
def get_costs(x1, x2, U, V, Q, Y, Z, T1, T2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += np.round(Manufacturing_costs[i][m]*Q[m,i])

    #grb_expr += pr_cost
    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += np.round(Supplier_cost[m][l]*V[m,l])
    #grb_expr += b_cost

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += np.round(lost_sales[k][m]*U[k,m])
    
    lst = [[Opening, in_house_shipping, outsourced_shipping, pr_cost, b_cost, l_cost]]
    print(OC_1 + OC_2 + ship_1 + ship_2 + ship_3 + ship_4 + pr_cost + b_cost + l_cost)
    return pd.DataFrame(lst, columns = ["Opening", "In House Shipping", "Outsourced Shipping", "Production", 
                         "Buying from Outsource", "Lost Sales"])

    
            
    

In [466]:
get_costs(v_val_x_i, v_val_x_j, v_val_U_km, v_val_V_lm, v_val_Q_im, v_val_Y_ijm,  v_val_Z_jkm,  v_val_T_ljm, v_val_T_lkm)

201.4523076855209


,Opening,In House Shipping,Outsourced Shipping,Production,Buying from Outsource,Lost Sales
0,45.0,105.0,0.0,51.0,0.0,0.0
